In [35]:
import pandas as pd
import pickle

In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [38]:
pd.__version__

'2.2.2'

In [39]:
import sklearn
sklearn.__version__
pickle.__version__

AttributeError: module 'pickle' has no attribute '__version__'

In [40]:
import sys
print(sys.version)

3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]


In [41]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [42]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [43]:
print("Q1: {columns:d}".format(columns = len(df1.columns)))

Q1: 19


In [44]:
df1["duration"] = (df1["tpep_dropoff_datetime"] - df1["tpep_pickup_datetime"])/pd.to_timedelta(1, unit="m")

In [45]:
print("Q2: {deviation:.2f}%".format(deviation = df1["duration"].std()))

Q2: 42.59%


In [46]:
    df1.duration.describe(percentiles=[0.01, 0.02, 0.95, 0.98, 0.99, 0.995])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
1%       7.833333e-01
2%       2.050000e+00
50%      1.151667e+01
95%      3.646667e+01
98%      4.873333e+01
99%      5.725000e+01
99.5%    6.551667e+01
max      1.002918e+04
Name: duration, dtype: float64

In [47]:
df = df1[ (df1["duration"]>=1) & (df1["duration"] <= 60)]

In [48]:
print("Q3: {fraction:.2f}%".format(fraction = (len(df)/len(df1))*100))

Q3: 98.12%


In [49]:
# sns.distplot(df.duration)

In [50]:
# df1.duration.describe(percentiles=[0.95, 0.98, 0.99, 0.999])

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

In [51]:
# df.head()

In [52]:
df = df.copy()

In [53]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [54]:
df[categorical] = df[categorical].astype(str)

In [55]:
# df.dtypes

In [56]:
# df[categorical + numerical].iloc[:10].to_dict(orient='records')

In [57]:
train_dict = df[categorical].to_dict(orient='records')

In [58]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

In [59]:
print("Q4: "+str(len(dv.feature_names_)))

Q4: 515


In [60]:
target = 'duration'
y_train = df[target].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [61]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [62]:
y_predict = lr.predict(X_train)

In [63]:
#sns.distplot(y_predict, label='prediction')
#sns.distplot(y_train, label='train')
#plt.legend

In [64]:
rmse = mean_squared_error(y_train, y_predict, squared=False)

/home/sir/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [65]:
"Q5: {rmse:.3f}".format(rmse = rmse)

'Q5: 7.649'

In [68]:
import pickle
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [67]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])/pd.to_timedelta(1, unit="m")
    df = df[ (df["duration"]>=1) & (df["duration"] <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [58]:
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
val_dicts = df_val[categorical].to_dict(orient='records')

In [59]:
X_val = dv.transform(val_dicts)

In [60]:
y_predict = lr.predict(X_val)
y_train = df_val[target].values

In [61]:
rmse = mean_squared_error(y_train, y_predict, squared=False)

In [62]:
"Q6: {rmse:.3f}".format(rmse = rmse)

'Q6: 7.812'